In [2]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from scipy.stats import kendalltau
import matplotlib.pyplot as plt
import random

In [3]:
def U_normalize(Adj):
    return normalize(Adj, norm='l1', axis=1)

def V_normalize(Adj):
    return normalize(Adj, norm='l1', axis=0)

In [61]:
dataset = ['abortion','gun'][0]

print(f"The dataset is {dataset}")

df_words_list = pd.read_parquet(f'output/{dataset}/list_of_words.parquet')
df_topics = pd.read_parquet(f'output/{dataset}/topics.parquet')
df_tweets = pd.read_parquet(f'output/{dataset}/tweets.parquet')
df_users = pd.read_parquet(f'output/{dataset}/uid_to_index.parquet')
df_label = pd.read_feather(f'data/{dataset}/allsides.feather')

The dataset is abortion


In [62]:
TW2U = sp.load_npz(f'output/{dataset}/Tweets2Users_sparse.npz')
TW2W = sp.load_npz(f'output/{dataset}/X_Tweets_sparse.npz')
TW2T = sp.load_npz(f'output/{dataset}/Tweets2Topics_sparse.npz')

TW2U.shape, TW2W.shape, TW2T.shape

((68810, 2041), (68810, 5890), (68810, 35))

In [63]:
words_list = df_words_list['word'].values

In [64]:
Users_Labels = np.load(f'output/{dataset}/users_labels_(Same_word_as_Tweets2Users).npy')
Users_Labels.shape

(2041,)

In [65]:
Users_Labels
user_ids = df_users['user_id'].values

uid2label = dict(zip(user_ids, Users_Labels))
len(uid2label)

2041

In [66]:
df_tweets['label'] = df_tweets['user_id'].map(uid2label)

x = df_tweets['label'].value_counts()
print(x)


tweet_label = df_tweets['label'].values
tweet_label.shape

label
0    34652
1    34158
Name: count, dtype: int64


(68810,)

# Keywords of Community:

In [67]:
T2TW = TW2T.T


# T -> TW -> W
T2TW_0 = T2TW[:, tweet_label == 0]
TW2W_0 = TW2W[tweet_label == 0, :]
T2TW_0.shape, TW2W_0.shape
PM_L = U_normalize(T2TW_0)
PM_R = V_normalize(TW2W_0)

HS_T2W_C0 = cosine_similarity(PM_L, PM_R.T)

In [68]:
T2TW_1 = T2TW[:, tweet_label == 1]
TW2W_1 = TW2W[tweet_label == 1, :]
PM_L = U_normalize(T2TW_1)
PM_R = V_normalize(TW2W_1)

HS_T2W_C1 = cosine_similarity(PM_L, PM_R.T)

In [69]:
topic_id = 3 ##!!!!!! Impotent
score_c0_topic = HS_T2W_C0[topic_id] - HS_T2W_C1[topic_id]
words_list[np.argsort(score_c0_topic)[::-1][:10]]

array(['feel', 'kidding', 'life', 'cking', 'thoughts', 'feeling', 'felt',
       'everytime', 'despair', 've'], dtype=object)

In [70]:
score_c1_topic = HS_T2W_C1[topic_id] - HS_T2W_C0[topic_id] ##!!!!!! Impotent
words_list[np.argsort(score_c1_topic)[::-1][:10]]

array(['ignorant', 'lmfao', 'wth', 'crap', 'idiots', 'stupid', 'anytime',
       'smug', 'jerk', 'outright'], dtype=object)

In [71]:
U2TW = TW2U.T
U2TW.shape

(2041, 68810)

In [72]:
TW2W.shape

(68810, 5890)

In [73]:
U2W = U2TW @ TW2W # User to Word matrix
U2W.shape

(2041, 5890)

# Another Way to compute the ideology:

In [74]:
top_50_words_c0 = np.argsort(score_c0_topic)[::-1][:30]
top_50_words_c1 = np.argsort(score_c1_topic)[::-1][:30]

In [75]:
top_50_words_c0

array([1973, 2879, 3043,  891, 5304, 1974, 1981, 1801, 1404, 5593, 2258,
       1846, 1867, 4804, 3052, 2500,  603, 5258, 2573, 1219, 3696, 4904,
       2483, 4851, 5825, 4178, 2887, 2160, 5461, 3380])

In [76]:
words_list[top_50_words_c0]

array(['feel', 'kidding', 'life', 'cking', 'thoughts', 'feeling', 'felt',
       'everytime', 'despair', 've', 'gotta', 'experiences', 'express',
       'silence', 'like', 'horrifying', 'boundaries', 'tend', 'idiot',
       'crying', 'pain', 'sorry', 'honestly', 'slow', 'worry', 'rage',
       'kind', 'fun', 'try', 'moment'], dtype=object)

In [77]:
top_50_words_c1

array([2580, 3088, 5852, 1171, 2575, 5092,  232, 4868, 2800, 3664, 3853,
       5524, 2077, 2714, 4791, 3930, 5005, 2196, 3040,  477, 1031, 2277,
       3968, 4817, 5458, 4120, 3149, 1190, 3185, 3523])

In [78]:
words_list[top_50_words_c1]

array(['ignorant', 'lmfao', 'wth', 'crap', 'idiots', 'stupid', 'anytime',
       'smug', 'jerk', 'outright', 'plain', 'unfortunately', 'fool',
       'intelligent', 'sickening', 'possibly', 'statement', 'gee', 'lie',
       'believe', 'conditioned', 'grandparents', 'premise', 'sir',
       'truth', 'pure', 'lunatic', 'cried', 'mama', 'nonsensical'],
      dtype=object)

In [79]:
Users_Labels.shape

(2041,)

In [80]:
c0_top50_words_names = words_list[top_50_words_c0]
c1_top50_words_names = words_list[top_50_words_c1]

In [81]:
df_tweets

,user_id,tweet,topic,label
0,1511561411696148480,Ok try not to Shit on yourself we forget your ...,3,1
1,1511561411696148480,Possibly using sperm and the egg from unvaccin...,6,1
2,1511561411696148480,Hear me out first. Movies always ahead of time...,6,1
3,1511561411696148480,I don t care if they get Brittany griner out o...,10,1
4,1511561411696148480,They would love to bankrupt our economy so the...,1,1
...,...,...,...,...
68805,987419369599676416,"If you read one thing today, this is it. What ...",3,1
68806,987419369599676416,Dis guy right here has the receipts.,0,1
68807,987419369599676416,So basically the three letter agencies were / ...,8,1
68808,987419369599676416,Right up until that LAST DEM BALLOTT DUMP.,2,1


In [82]:
tweets_topic_mask = (df_tweets.topic == topic_id).to_numpy()
tweets_topic_mask

array([ True, False, False, ..., False, False, False])

In [83]:
U2TW_topic = U2TW[:, tweets_topic_mask]
U2TW_topic.shape

(2041, 4409)

In [84]:
TW2W_topic = TW2W[tweets_topic_mask, :]
TW2W_topic.shape

(4409, 5890)

In [85]:
# U2W = U2TW @ TW2W # User to Word matrix
U2W_topic = U2TW_topic @ TW2W_topic
U2W_topic.shape

(2041, 5890)

# Ideology:

In [86]:
df_label

,user_id,allsides_score
0,24908953,-0.125000
1,1340694200770154496,-0.500000
2,576828226,0.000000
3,1424482039974203399,-0.187500
4,165793361,-0.464286
...,...,...
1994,341190477,-0.500000
1995,197535254,-0.473684
1996,2856205943,-0.600000
1997,971009759125131264,-0.500000


In [87]:
df_users

,user_id,user_index
0,1511561411696148480,0
1,1519024813561368576,1
2,1537105274757976065,2
3,438897163,3
4,55712928,4
...,...,...
2036,1489362675889065985,2036
2037,1424448523244871688,2037
2038,438584384,2038
2039,999149146429906944,2039


In [88]:
df_users = df_users.merge(df_label, on='user_id', how='left')
df_users

,user_id,user_index,allsides_score
0,1511561411696148480,0,NaN
1,1519024813561368576,1,0.666667
2,1537105274757976065,2,0.000000
3,438897163,3,-0.576923
4,55712928,4,NaN
...,...,...,...
2036,1489362675889065985,2036,NaN
2037,1424448523244871688,2037,NaN
2038,438584384,2038,NaN
2039,999149146429906944,2039,NaN


In [89]:
ideology = df_users['allsides_score'].values
ideology

array([       nan, 0.66666667, 0.        , ...,        nan,        nan,
              nan])

____

# For all the topics

In [102]:
tweet_topic = df_tweets['topic'].values

In [103]:
tweet_topic

array([ 3,  6,  6, ...,  8,  2, 24])

In [104]:
k = 30 # Number of top words to  use to compute ideology

p = []
corr = []

for topic_id in range(len(df_topics)):
    topic_name = df_topics['Name'][topic_id]
    
    # keep the tweets about the topic
    U2TW_t = U2TW[:, tweet_topic == topic_id]
    TW2W_t = TW2W[tweet_topic == topic_id, :]
    
    U2W_t = U2TW_t @ TW2W_t
    
    U2W_bin_t = U2W_t.copy()
    U2W_bin_t[U2W_bin_t > 0] = 1
    
    U2W_bin_0_t = U2W_bin_t[Users_Labels == 0, :]
    U2W_bin_1_t = U2W_bin_t[Users_Labels == 1, :]
    
    ratio_0 = U2W_bin_0_t.sum(axis=0) / U2W_bin_0_t.shape[0]
    ratio_1 = U2W_bin_1_t.sum(axis=0) / U2W_bin_1_t.shape[0]
    
    ratio_0 = ratio_0.A1
    ratio_1 = ratio_1.A1
    
    
    score_c0_topic = (HS_T2W_C0[topic_id] - HS_T2W_C1[topic_id]) * ratio_0
    score_c1_topic = (HS_T2W_C1[topic_id] - HS_T2W_C0[topic_id]) * ratio_1
    
    top_k_words_c0 = np.argsort(score_c0_topic)[::-1][:k]
    top_k_words_c1 = np.argsort(score_c1_topic)[::-1][:k]
    
    print(f"Topic {topic_id}: {topic_name}")
    
    
    user_scores = []
    ground_truth = []

    for i in range(U2W_t.shape[0]):
        if np.isnan(ideology[i]):
            # print('nan')
            continue
        
        user_words = U2W_t[i].toarray().squeeze()
        
        # idx = Users_Labels[i] 
        
        user_words_c0 = user_words[top_k_words_c0]
        user_words_c1 = user_words[top_k_words_c1]
        
        # Count non zero words 
        non_zero_c0 = sum(user_words_c0 > 0)
        non_zero_c1 = sum(user_words_c1 > 0)
        
        if (non_zero_c0 + non_zero_c1) <= 3:
            continue
        
        s0 = sum(user_words_c0)
        s1 = sum(user_words_c1)
        score = (s0 - s1) / (s0 + s1)
        
        # user_scores[idx].append(score)
        
        user_scores.append(score)
        ground_truth.append(ideology[i])
        
    user_scores = np.array(user_scores)
    ground_truth = np.array(ground_truth)
    
    if len(user_scores) < 10:
        print(f"number of users: {len(user_scores)} is less than 10")
        continue
    
    print(f"number of users: {len(user_scores)}")
    # print(f"Kendall tau: {kendalltau(user_scores, -ground_truth)}")
    print(f"Spearman correlation: {spearmanr(user_scores, -ground_truth)}")
    
    corr_, p_ = spearmanr(user_scores, -ground_truth)
    if len(user_scores) >= 100:
        p.append(p_)
        corr.append(corr_)
    
    print("\n\n")
    

Topic 0: 0_fbi_trump_documents_doj
number of users: 439
Spearman correlation: SignificanceResult(statistic=0.5946181551447819, pvalue=2.5366509013952266e-43)



Topic 1: 1_climate_inflation_energy_gas
number of users: 336
Spearman correlation: SignificanceResult(statistic=0.39732869918151376, pvalue=3.7347173971551736e-14)



Topic 2: 2_vote_democrats_gop_republicans
number of users: 302
Spearman correlation: SignificanceResult(statistic=0.3995922821721817, pvalue=5.26387249646286e-13)



Topic 3: 3_know_people_just_don
number of users: 140
Spearman correlation: SignificanceResult(statistic=0.26565042335021577, pvalue=0.001512699803550849)



Topic 4: 4_abortion_roe_abortions_pro
number of users: 252
Spearman correlation: SignificanceResult(statistic=0.45247579136107796, pvalue=4.022656612300048e-14)



Topic 5: 5_god_jesus_church_lord
number of users: 120
Spearman correlation: SignificanceResult(statistic=0.4936005772404741, pvalue=1.0140007519573905e-08)



Topic 6: 6_covid_vaccine_p

In [105]:
len(p), len(corr)

(15, 15)

In [106]:
p

[2.5366509013952266e-43,
 3.7347173971551736e-14,
 5.26387249646286e-13,
 0.001512699803550849,
 4.022656612300048e-14,
 1.0140007519573905e-08,
 2.2067600999849094e-08,
 2.3689171694533122e-11,
 3.775134208221122e-09,
 5.9485771351348615e-08,
 4.339771655182375e-06,
 2.996394390708509e-06,
 4.527090819210431e-11,
 8.738187024091874e-13,
 0.0027343269458630524]

In [107]:
corr

[0.5946181551447819,
 0.39732869918151376,
 0.3995922821721817,
 0.26565042335021577,
 0.45247579136107796,
 0.4936005772404741,
 0.39502926869461824,
 0.47388890582466653,
 0.4436138924989046,
 0.3524454290327377,
 0.3584325015816696,
 0.41006984531718604,
 0.5032501308353633,
 0.47643262808201237,
 0.2843079150171723]

In [108]:
np.mean(corr), np.mean(p)

(0.42004909635563836, 0.0002836305636274487)